# Imports

In [1]:
import warnings

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import datasets, linear_model
from sklearn import tree
import sklearn.cluster as cluster
from sklearn.metrics import accuracy_score
import plotly.express as px
import plotly.graph_objects as go

np.random.seed(1234)

In [2]:
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)
plt.rcParams.update({"axes.grid": True, "figure.figsize": (8, 4)})
import seaborn as sns

# sns.set()

# 15.1 

In [3]:
# housing = datasets.fetch_california_housing()
# df = pd.DataFrame(data=housing.data, columns=housing.feature_names)
# df = pd.read_csv("./data/housing.csv")
# display(df)

## 

